<ul>
    <li> <h2><b>Universidade de Brasília - UnB</b></h2> </li>
    <li> <b>Campus UnB Gama</b> </li>
    <li> <b>Disciplina: Processamento Digital de Sinais Financeiros</b> </li>
    <a href="https://www.linkedin.com/in/marcelino-andrade-b164b369/">
    <li> <b>Prof. Marcelino Monteiro de Andrade Dr.</b> </li>
</ul>


    Primeira (1/5) Atividade Prática Avaliativa
    Entrega da Atividade 01/10/2019 
    "A não entrega na data esperada reduz em 50% na nota da avaliação"


<div id="Tópicos">
    <h2>Questões</h2>
</div>


<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol> 
        <li><a href="#Tópico 0">Questão 1</a></li>
        <li><a href="#Tópico 1">Questão 2</a></li>
        <li><a href="#Tópico 2">Questão 3</a></li>
        <li><a href="#Tópico 3">Questão 4</a></li>

</ol>
</div>


<div id="Tópico 0">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 1: Considerando os índices do grupo Setorial ou por Segmento, escolha 5 (cinco) índices e colete os preços diários (OHLC), assim como os volumes, dos 3 (três) papéis mais negociados (líquidos) em cada um dos 5 índices durante um período de 24 meses.
</h></a></h>   
</div> </div> 
http://www.bmfbovespa.com.br/pt_br/produtos/indices/

In [1]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mpl_finance import candlestick_ohlc
import datetime
import pandas_datareader.data as pdr
import pickle
import datetime as dt
from scipy import signal


In [4]:
import matplotlib.dates as mdates
import datetime
from pandas import *
import matplotlib.mlab as mlab
import scipy.signal as signal
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
import yfinance as yf
import datetime as dt

def yahoo(symbols,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year):
    yf.pdr_override()       
    stocks_yahoo = pdr.get_data_yahoo(symbols, start=datetime(S_Year, S_Month, S_Day), end=datetime(E_Year, E_Month, E_Day),   as_panel = False,)
    return stocks_yahoo

def retira_colunas_nan(df):
    newdf = None
    for i in range(len(df['Close'].columns)):
        if df['Close'].iloc[:,i].isna().sum() < 100:
            newdf=pd.concat([newdf,df['Close'].iloc[:,i]],axis=1) 
    return newdf

def ordena_acoes_por_diferenca_de_preco(df):
    resultado = {}
    for i in range(len(df.columns)):
        diferenca = df.iloc[-1,i] - df.iloc[0,i]
        resultado.update({df.iloc[0:i].columns[i]: diferenca})
        
    dict_ordered = sorted(resultado.items(), key=lambda x: x[1], reverse=True)
    
    for i in dict_ordered.value:
        if dict_ordered[i]<0:
            del dict_ordered[i]
            
    print(dict_ordered)

def cria_stocks_all(df):
    Stocks=df.values[:-2]
    N_months=24
    Stocks=[Stocks[k]+'.SA' for k in range(len(Stocks))]
    
    today = dt.date.today()
    E_Day = today.day; E_Month = today.month; E_Year = today.year
    day_before=today - dt.timedelta(days=N_months*365/12)
    S_Day = day_before.day; S_Month = day_before.month; S_Year = day_before.year

    stocks_all=yahoo(Stocks,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year)

    return stocks_all

def seleciona_acoes_crescentes(df):
    resultado = {}
    for i in range(len(df.columns)):
        diferenca = df.iloc[-1,i] - df.iloc[0,i]
        if diferenca > 0:
            resultado.update({df.iloc[0:i].columns[i]: diferenca})
        
    dict_crescentes = sorted(resultado.items(), key=lambda x: x[1], reverse=True) 

    return dict_crescentes

def seleciona_ativos_por_volume(dict_crescentes, stocks_all):
    dict_volume = {}
    for i in range(len(dict_crescentes)):
        dict_volume.update({dict_crescentes[i][0]: stocks_all['Volume'][dict_crescentes[i][0]].sum()}) 

    dict_volume_ordered = sorted(dict_volume.items(), key=lambda x: x[1], reverse=True) 
    dict_volume_ordered[:3]
    
    nome_acoes = []
    for i in dict_volume_ordered[:3]:
        nome_acoes.append(i[0])
        
    return nome_acoes


In [5]:
#Os dados foram baixados no site http://www.bmfbovespa.com.br/pt_br/produtos/indices/

# Lendo dados do indice IMOB
dfIMOB = pd.read_csv('C:/Users/bruno/Desktop/IMOBdata.csv')
dfIMOB = dfIMOB['Código\xa0']

stocks_all_IMOB = cria_stocks_all(dfIMOB)
stocks_all_IMOB_clean = retira_colunas_nan(stocks_all_IMOB)
dict_crescentes_IMOB = seleciona_acoes_crescentes(stocks_all_IMOB_clean)
nome_acoes_IMOB = seleciona_ativos_por_volume(dict_crescentes_IMOB, stocks_all_IMOB)

df_selecionadas_IMOB = pd.DataFrame()
for i in range(len(nome_acoes_IMOB)):
    df_selecionadas_IMOB = pd.concat([df_selecionadas_IMOB,stocks_all_IMOB['Close'][nome_acoes_IMOB[i]]],axis=1) 
    
df_selecionadas_IMOB

[*********************100%***********************]  14 of 14 downloaded


,BRML3.SA,MRVE3.SA,CYRE3.SA
2017-09-26,14.30,12.56,13.79
2017-09-27,14.10,12.38,13.39
2017-09-28,13.90,12.21,13.37
2017-09-29,14.07,12.51,13.80
2017-10-02,14.09,12.45,13.76
2017-10-03,14.39,12.77,14.12
2017-10-04,14.30,12.96,14.07
2017-10-05,13.94,12.67,14.33
2017-10-06,13.80,12.71,14.01
2017-10-09,13.90,12.75,13.80


In [9]:
# Lendo dados do indice IDIV
dfIDIV = pd.read_csv('C:/Users/bruno/Desktop/IDIVdata.csv')
dfIDIV = dfIDIV['Código\xa0']

stocks_all_IDIV = cria_stocks_all(dfIDIV)
stocks_all_IDIV_clean = retira_colunas_nan(stocks_all_IDIV)
dict_crescentes_IDIV = seleciona_acoes_crescentes(stocks_all_IDIV_clean)
nome_acoes_IDIV = seleciona_ativos_por_volume(dict_crescentes_IDIV,stocks_all_IDIV)

df_selecionadas_IDIV = pd.DataFrame()
for i in range(len(nome_acoes_IDIV)):
     df_selecionadas_IDIV = pd.concat([df_selecionadas_IDIV,stocks_all_IDIV['Close'][nome_acoes_IDIV[i]]],axis=1) 
    
df_selecionadas_IDIV


[*********************100%***********************]  31 of 31 downloaded


,ITSA4.SA,ITUB4.SA,CMIG4.SA
2017-09-26,10.00,28.79,7.80
2017-09-27,9.85,28.81,7.68
2017-09-28,9.84,28.65,7.80
2017-09-29,10.03,28.90,7.74
2017-10-02,9.96,28.94,7.72
2017-10-03,10.35,29.93,8.16
2017-10-04,10.28,29.67,7.96
2017-10-05,10.35,30.14,8.13
2017-10-06,10.30,29.79,8.01
2017-10-09,10.15,29.43,7.99


In [25]:
# Lendo dados do indice ICON
dfICON = pd.read_csv('C:/Users/bruno/Desktop/ICONdata.csv')
dfICON = dfICON['Código\xa0']

stocks_all_ICON = cria_stocks_all(dfICON)
stocks_all_ICON_clean = retira_colunas_nan(stocks_all_ICON)
dict_crescentes_ICON = seleciona_acoes_crescentes(stocks_all_ICON_clean)
nome_acoes_ICON = seleciona_ativos_por_volume(dict_crescentes_ICON, stocks_all_ICON)

df_selecionadas_ICON = pd.DataFrame()
for i in range(len(nome_acoes_ICON)):
    df_selecionadas_ICON = pd.concat([df_selecionadas_ICON,stocks_all_ICON['Close'][nome_acoes_ICON[i]]],axis=1) 
    
df_selecionadas_ICON


[*********************100%***********************]  54 of 54 downloaded


,MGLU3.SA,VVAR3.SA,JBSS3.SA
2017-09-26,9.25,7.05,8.38
2017-09-27,8.54,6.89,8.45
2017-09-28,8.84,7.28,8.25
2017-09-29,9.28,7.84,8.50
2017-10-02,9.73,7.99,8.67
2017-10-03,9.87,8.19,8.80
2017-10-04,9.94,8.35,8.78
2017-10-05,9.80,8.05,8.77
2017-10-06,9.96,8.18,8.57
2017-10-09,9.72,8.02,8.42


In [15]:
# Lendo dados do indice IEE
dfIEE = pd.read_csv('C:/Users/bruno/Desktop/IEEdata.csv')
dfIEE = dfIEE['Código\xa0']

stocks_all_IEE = cria_stocks_all(dfIEE)
stocks_all_IEE_clean = retira_colunas_nan(stocks_all_IEE)
dict_crescentes_IEE = seleciona_acoes_crescentes(stocks_all_IEE_clean)
nome_acoes_IEE = seleciona_ativos_por_volume(dict_crescentes_IEE, stocks_all_IEE)

df_selecionadas_IEE = pd.DataFrame()
for i in range(len(nome_acoes_IEE)):
    df_selecionadas_IEE = pd.concat([df_selecionadas_IEE,stocks_all_IEE['Close'][nome_acoes_IEE[i]]],axis=1) 
    
df_selecionadas_IEE


[*********************100%***********************]  15 of 15 downloaded


,CMIG4.SA,ELET3.SA,ENBR3.SA
2017-09-26,7.80,20.49,15.21
2017-09-27,7.68,19.90,15.33
2017-09-28,7.80,19.55,15.00
2017-09-29,7.74,19.76,15.14
2017-10-02,7.72,20.54,15.00
2017-10-03,8.16,21.80,15.16
2017-10-04,7.96,21.85,15.37
2017-10-05,8.13,21.47,15.40
2017-10-06,8.01,20.81,15.35
2017-10-09,7.99,20.72,15.24


In [23]:
# Lendo dados do indice INDX
dfINDX = pd.read_csv('C:/Users/bruno/Desktop/INDXdata.csv')
dfINDX = dfINDX['Código\xa0']

stocks_all_INDX = cria_stocks_all(dfINDX)
stocks_all_INDX_clean = retira_colunas_nan(stocks_all_INDX)
dict_crescentes_INDX = seleciona_acoes_crescentes(stocks_all_INDX_clean)
nome_acoes_INDX = seleciona_ativos_por_volume(dict_crescentes_INDX, stocks_all_INDX)

df_selecionadas_INDX = pd.DataFrame()
for i in range(len(nome_acoes_INDX)):
    df_selecionadas_INDX = pd.concat([df_selecionadas_INDX,stocks_all_INDX['Close'][nome_acoes_INDX[i]]],axis=1) 
    
df_selecionadas_INDX


[*********************100%***********************]  41 of 41 downloaded


,GOAU4.SA,GGBR4.SA,CSNA3.SA
2017-09-26,5.30,11.11,9.49
2017-09-27,5.22,11.16,9.27
2017-09-28,5.12,10.86,9.08
2017-09-29,5.29,11.04,9.61
2017-10-02,5.33,11.00,9.88
2017-10-03,5.62,11.40,10.86
2017-10-04,5.70,11.37,10.76
2017-10-05,5.45,11.08,10.53
2017-10-06,5.34,10.76,10.25
2017-10-09,5.37,10.70,9.90


In [26]:
df_selecionadas = pd.concat([df_selecionadas_IEE,df_selecionadas_INDX,df_selecionadas_ICON,df_selecionadas_IDIV,df_selecionadas_IMOB],axis=1)

In [28]:
df_selecionadas.corr()

,CMIG4.SA,ELET3.SA,ENBR3.SA,GOAU4.SA,GGBR4.SA,CSNA3.SA,MGLU3.SA,VVAR3.SA,JBSS3.SA,ITSA4.SA,ITUB4.SA,CMIG4.SA,BRML3.SA,MRVE3.SA,CYRE3.SA
CMIG4.SA,1.000000,0.911456,0.846694,0.128845,0.048911,0.796580,0.862316,-0.517125,0.806505,0.776107,0.783318,1.000000,0.649466,0.633584,0.794003
ELET3.SA,0.911456,1.000000,0.924734,-0.065033,-0.151499,0.812457,0.829180,-0.223562,0.883546,0.778213,0.741641,0.911456,0.767976,0.794640,0.919601
ENBR3.SA,0.846694,0.924734,1.000000,-0.178404,-0.250282,0.872556,0.807143,-0.207182,0.907132,0.615124,0.564308,0.846694,0.702212,0.836820,0.871938
GOAU4.SA,0.128845,-0.065033,-0.178404,1.000000,0.975217,0.015945,0.150028,-0.301408,-0.099456,0.345227,0.376404,0.128845,-0.369181,-0.114115,-0.101252
GGBR4.SA,0.048911,-0.151499,-0.250282,0.975217,1.000000,-0.049616,0.064055,-0.276140,-0.171850,0.286891,0.310692,0.048911,-0.443563,-0.169801,-0.182641
CSNA3.SA,0.796580,0.812457,0.872556,0.015945,-0.049616,1.000000,0.714726,-0.299984,0.831141,0.588085,0.553590,0.796580,0.552587,0.782448,0.769730
MGLU3.SA,0.862316,0.829180,0.807143,0.150028,0.064055,0.714726,1.000000,-0.327772,0.899531,0.628573,0.597008,0.862316,0.474424,0.699570,0.825766
VVAR3.SA,-0.517125,-0.223562,-0.207182,-0.301408,-0.276140,-0.299984,-0.327772,1.000000,-0.151610,-0.125979,-0.219944,-0.517125,0.064923,0.170781,0.045441
JBSS3.SA,0.806505,0.883546,0.907132,-0.099456,-0.171850,0.831141,0.899531,-0.151610,1.000000,0.606050,0.533480,0.806505,0.590139,0.863895,0.909851
ITSA4.SA,0.776107,0.778213,0.615124,0.345227,0.286891,0.588085,0.628573,-0.125979,0.606050,1.000000,0.979189,0.776107,0.639804,0.641655,0.762006


In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

N_months=60
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jan 29 09:11:31 2018

@author: Prof Marcelino Andrade
Universidade de Brasilia - UnB
"""

import matplotlib.dates as mdates
import datetime
from pandas import *
import matplotlib.mlab as mlab
import scipy.signal as signal
import matplotlib.pyplot as plt
import pandas_datareader.data as pdr
import yfinance as yf
import datetime as dt

sns.set()

N_months=48

output = None


df = pd.read_html('http://www.bmfbovespa.com.br/pt_br/produtos/indices/indices-setoriais/indice-bm-fbovespa-energia-eletrica-iee-composicao-da-carteira.htm')[0]['Código']
print(len(df.values))
output = df          

output=output.dropna()

symbols=list(output.values)

Stocks=[]
for symbol in symbols:
    if len(symbol)==5:
        Stocks.append(symbol)

Stock=[]
Stocks=[Stocks[k]+'.SA' for k in range(len(Stocks))]
#Stocks.append('^BVSP')


today = dt.date.today()
E_Day = today.day; E_Month = today.month; E_Year = today.year
day_before=today - dt.timedelta(days=N_months*365/12)
S_Day = day_before.day; S_Month = day_before.month; S_Year = day_before.year

stocks_all=yahoo(Stocks,S_Day, S_Month, S_Year, E_Day, E_Month, E_Year)

stocks_close=stocks_all['Close'].dropna(axis=0, how='all')
stocks_close=stocks_close.dropna(axis=1, how='any')
stocks_open=stocks_all['Open'].dropna(axis=0, how='all')
stocks_open=stocks_open.dropna(axis=1, how='any')
stocks_high=stocks_all['High'].dropna(axis=0, how='all')
stocks_high=stocks_high.dropna(axis=1, how='any')
stocks_low=stocks_all['Low'].dropna(axis=0, how='all')
stocks_low=stocks_low.dropna(axis=1, how='any')

output = None

symbols=list(stocks_close.columns)

for sy in symbols:

    if output is None:
        df = pd.DataFrame(index=stocks_open.index)
        df[sy[:-3]]=stocks_close[sy]
        output = df          

    else:
        df = pd.DataFrame(index=stocks_open.index)
        df[sy[:-3]]=stocks_close[sy]      
        output=pd.concat([output,df],axis=1)

41 1
41 2
41 3
41 4
41 5
41 6
41 7
41 8
41 9
41 10
41 11
40 12
[*********************100%***********************]  204 of 204 downloaded


<div id="Tópico 1">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 2: Identifique a correlação entre os preços e os retornos dos 15 (quinze) ativos coletados na questão anterior. Em seguida, identifique os cinco papeis com menor correlação e retorno médio positivo no período de 24 meses.</h></a></h>   
</div> </div> 

In [2]:
a=1

<div id="Tópico 2">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 3: Com relação aos 5 (cinco) papéis da questão 2, apresente: (i) os gráficos de preço no formato candlestick, (ii) os gráficos de retorno mês, (iii) o índice Sharpe e o (iv) fator Beta com relação ao índice Bovespa.</h></a></h>   
</div> </div> 

In [3]:
a=1

<div id="Tópico 3">
    <div class="alert alert-block alert-info" style="margin-top: 0px">
        <h><a href="#Tópicos"><h>1) Questão 4: Considerando os 5 (cinco) papeis escolhidos, na questão 2, aplique a Teoria do Portfólio Moderno de Markowitz. Nesse sentido, (i) Identifique o diagrama Risco x Retorno, (ii) a curva de eficiência, (iii) o portfólio de menor risco e o (iv) portfólio com maior índice Sharpe.</h></a></h>   
</div> </div> 

In [4]:
a=1